Install transformers add upgrade transformers:

In [ ]:
!pip install transformers torch accelerate bitsandbytes sentencepiece gradio
!pip install --upgrade transformers

This is the full import section:

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, pipeline
from huggingface_hub import login
from google.colab import userdata
import gradio as gr

Hugging Face Login, modelid and config, put the secret to HF_TOKEN in your notebook secret, please remeber to request the llama model files access permission.

In [ ]:
#print(userdata.get('HF_TOKEN'))
login(token=userdata.get('HF_TOKEN'), add_to_git_credential=True)


#model_id = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
model_id = 'Qwen/Qwen2-7B-Instruct'

#quantization_config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

config = AutoConfig.from_pretrained(model_id)
config.rope_scaling = { "type": "linear", "factor": 2.0 }  # Adjust the factor as needed

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto')
#model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto')



Text generator:

In [ ]:
text_generator = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
)

Get response:

In [ ]:
def get_response(prompt):
  response = text_generator(prompt)
  return response[0]['generated_text']

Give a test:

In [ ]:
prompt = "write a java code, output the difference between current time and 2001/7/1"
response = get_response(prompt)
print(response)

Add web GUI access

In [ ]:
# use Gradio to create web
def gradio_interface(prompt):
    return get_response(prompt)


interface = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs="text",
    title="LLM inference 8b in notebook",
    description="please input the prompt text."
)

# start it up
interface.launch(share=True)

Clean up the resources if necessary

In [9]:
del model  # remove model
del tokenizer # remove tokenizer
torch.cuda.empty_cache()


**Conclusion:**

tried two models 'meta-llama/Meta-Llama-3.1-8B-Instruct' and 'Qwen/Qwen2-7B-Instruct'.


*   both of them should be quantized in google colab 16G GPU
*   Qwen2's performance is better than Llama3.1 in Chinese
*   It seems Qwen2 is faster than Llama3.1, 25 seconds v.s. 1.5 minutes. But both of them are slow, maybe the stream output will have a better uer experence.
*   Llama3.1 will repeat the output, maybe need to be optimized.
*   After quantization, the memory usage around 4G, the GPU usage around 6G.

